# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 13 2022 10:46AM,248546,19,ADV80006507,"AdvaGen Pharma, Ltd",Released
1,Oct 13 2022 10:45AM,248545,10,WT5377 Out,Eywa Pharma Inc.,Released
2,Oct 13 2022 10:42AM,248544,20,8641364,"Emersa Waterbox, LLC",Released
3,Oct 13 2022 10:38AM,248543,19,TR-RN-10122022,"GCH Granules USA, Inc.",Released
4,Oct 13 2022 10:35AM,248266,15,PNC391300B,"Person & Covey, Inc.",Released
5,Oct 13 2022 10:35AM,248266,15,PNC391301B,"Person & Covey, Inc.",Released
6,Oct 13 2022 10:35AM,248266,15,PNC391306B,"Person & Covey, Inc.",Released
7,Oct 13 2022 10:35AM,248266,15,PNC394476A,"Person & Covey, Inc.",Released
8,Oct 13 2022 10:35AM,248266,15,PNC394478A,"Person & Covey, Inc.",Released
9,Oct 13 2022 10:35AM,248266,15,PNC394479A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,248541,Released,3
27,248543,Released,1
28,248544,Released,1
29,248545,Released,1
30,248546,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
248541,NaN,3.0
248543,NaN,1.0
248544,NaN,1.0
248545,NaN,1.0
248546,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248083,1.0,0.0
248087,0.0,1.0
248089,0.0,1.0
248091,0.0,1.0
248266,0.0,30.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248083,1,0
248087,0,1
248089,0,1
248091,0,1
248266,0,30


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248083,1,0
1,248087,0,1
2,248089,0,1
3,248091,0,1
4,248266,0,30


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248083,1,
1,248087,,1
2,248089,,1
3,248091,,1
4,248266,,30


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 13 2022 10:46AM,248546,19,"AdvaGen Pharma, Ltd"
1,Oct 13 2022 10:45AM,248545,10,Eywa Pharma Inc.
2,Oct 13 2022 10:42AM,248544,20,"Emersa Waterbox, LLC"
3,Oct 13 2022 10:38AM,248543,19,"GCH Granules USA, Inc."
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc."
34,Oct 13 2022 10:31AM,248540,15,"Alliance Pharma, Inc."
79,Oct 13 2022 10:28AM,248541,10,Emerginnova
82,Oct 13 2022 10:27AM,248539,10,"Snap Medical Industries, LLC"
84,Oct 13 2022 10:20AM,248538,10,Bio-PRF
101,Oct 13 2022 10:19AM,248537,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 13 2022 10:46AM,248546,19,"AdvaGen Pharma, Ltd",,1
1,Oct 13 2022 10:45AM,248545,10,Eywa Pharma Inc.,,1
2,Oct 13 2022 10:42AM,248544,20,"Emersa Waterbox, LLC",,1
3,Oct 13 2022 10:38AM,248543,19,"GCH Granules USA, Inc.",,1
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",,30
5,Oct 13 2022 10:31AM,248540,15,"Alliance Pharma, Inc.",,45
6,Oct 13 2022 10:28AM,248541,10,Emerginnova,,3
7,Oct 13 2022 10:27AM,248539,10,"Snap Medical Industries, LLC",,2
8,Oct 13 2022 10:20AM,248538,10,Bio-PRF,,17
9,Oct 13 2022 10:19AM,248537,10,Eywa Pharma Inc.,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:46AM,248546,19,"AdvaGen Pharma, Ltd",1,
1,Oct 13 2022 10:45AM,248545,10,Eywa Pharma Inc.,1,
2,Oct 13 2022 10:42AM,248544,20,"Emersa Waterbox, LLC",1,
3,Oct 13 2022 10:38AM,248543,19,"GCH Granules USA, Inc.",1,
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",30,
5,Oct 13 2022 10:31AM,248540,15,"Alliance Pharma, Inc.",45,
6,Oct 13 2022 10:28AM,248541,10,Emerginnova,3,
7,Oct 13 2022 10:27AM,248539,10,"Snap Medical Industries, LLC",2,
8,Oct 13 2022 10:20AM,248538,10,Bio-PRF,17,
9,Oct 13 2022 10:19AM,248537,10,Eywa Pharma Inc.,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:46AM,248546,19,"AdvaGen Pharma, Ltd",1,
1,Oct 13 2022 10:45AM,248545,10,Eywa Pharma Inc.,1,
2,Oct 13 2022 10:42AM,248544,20,"Emersa Waterbox, LLC",1,
3,Oct 13 2022 10:38AM,248543,19,"GCH Granules USA, Inc.",1,
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",30,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:46AM,248546,19,"AdvaGen Pharma, Ltd",1.0,NaN
1,Oct 13 2022 10:45AM,248545,10,Eywa Pharma Inc.,1.0,NaN
2,Oct 13 2022 10:42AM,248544,20,"Emersa Waterbox, LLC",1.0,NaN
3,Oct 13 2022 10:38AM,248543,19,"GCH Granules USA, Inc.",1.0,NaN
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",30.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:46AM,248546,19,"AdvaGen Pharma, Ltd",1.0,0.0
1,Oct 13 2022 10:45AM,248545,10,Eywa Pharma Inc.,1.0,0.0
2,Oct 13 2022 10:42AM,248544,20,"Emersa Waterbox, LLC",1.0,0.0
3,Oct 13 2022 10:38AM,248543,19,"GCH Granules USA, Inc.",1.0,0.0
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",30.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3479267,104.0,9.0
15,496806,75.0,0.0
18,248462,0.0,1.0
19,1491136,5.0,3.0
20,248544,1.0,0.0
21,992350,3.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3479267,104.0,9.0
1,15,496806,75.0,0.0
2,18,248462,0.0,1.0
3,19,1491136,5.0,3.0
4,20,248544,1.0,0.0
5,21,992350,3.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,104.0,9.0
1,15,75.0,0.0
2,18,0.0,1.0
3,19,5.0,3.0
4,20,1.0,0.0
5,21,3.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,104.0
1,15,Released,75.0
2,18,Released,0.0
3,19,Released,5.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20,21
Status,,,,,,
Executing,9.0,0.0,1.0,3.0,0.0,1.0
Released,104.0,75.0,0.0,5.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20,21
0,Executing,9.0,0.0,1.0,3.0,0.0,1.0
1,Released,104.0,75.0,0.0,5.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20,21
0,Executing,9.0,0.0,1.0,3.0,0.0,1.0
1,Released,104.0,75.0,0.0,5.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()